<a href="https://colab.research.google.com/github/LexDF/votkai/blob/main/NotEasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# #@title Environment Setup

#@markdown ---
#@markdown # Civitai API Keyの設定(HunyuanVideo LoRAのダウンロードに必要です)
#@markdown Civitaiにホストされているモデルをダウンロードする場合はAPIキーが必要になりました。APIキーはCivitaiのサイトにログインした状態で以下のリンクから取得できます。

#@markdown https://civitai.com/user/account

#@markdown **なおCivitaiのAPI Keyは人と共有しないようにお気をつけください**

civitai_api_key = "d9c3085632fb8f24c8561b7068bb3b7f" #@param {type:"string"}

#@markdown ---

### setup ComfyUI ###
from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

### setup  ###
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors -O models/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors -O models/text_encoders/clip_l.safetensors
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors -O models/text_encoders/llava_llama3_fp8_scaled.safetensors
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/vae/hunyuan_video_vae_bf16.safetensors -O models/vae/hunyuan_video_vae_bf16.safetensors


#@markdown ---
#@markdown ## LoRA

#@markdown ### Pixel style hunyuan video model
#@markdown [公式ページ](https://civitai.com/models/1144459/pixel-style-hunyuan-video-model?modelVersionId=1287140)
use_pixel_style_hunyuan_video_model = True #@param {type: "boolean"}
if use_pixel_style_hunyuan_video_model:
  !wget https://civitai.com/api/download/models/1287140?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### Studio Ghibli Style 🎞️ HunyuanVideo
#@markdown [公式ページ](https://civitai.com/models/1084814/studio-ghibli-style-hunyuanvideo?modelVersionId=1218122)
use_studio_ghibli_style_hunyuan_video = True #@param {type: "boolean"}
if use_studio_ghibli_style_hunyuan_video:
  !wget https://civitai.com/api/download/models/1218122?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### Flat Color - Style
#@markdown [公式ページ](https://civitai.com/models/1132089/flat-color-style?modelVersionId=1315010)
use_flat_color_style = True #@param {type: "boolean"}
if use_flat_color_style:
  !wget https://civitai.com/api/download/models/1315010?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### HunyuanVideo - Frost Bloom 霜华
#@markdown [公式ページ](https://civitai.com/models/1068416/hunyuanvideo-frost-bloom?modelVersionId=1199182)
use_hunyuan_video_frost_bloom = True #@param {type: "boolean"}
if use_hunyuan_video_frost_bloom:
  !wget https://civitai.com/api/download/models/1199182?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### Minrill Style Animation - Hunyuan Video LoRA
#@markdown [公式ページ](https://civitai.com/models/1120207/minrill-style-animation-hunyuan-video-lora?modelVersionId=1258988)
use_minrill_style_animation_hunyuan_video_lora = False #@param {type: "boolean"}
if use_minrill_style_animation_hunyuan_video_lora:
  !wget https://civitai.com/api/download/models/1258988?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### underground club
#@markdown [公式ページ](https://civitai.com/models/314278/underground-club?modelVersionId=1237165)
use_underground_club = True #@param {type: "boolean"}
if use_underground_club:
  !wget https://civitai.com/api/download/models/1237165?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### CoolGuyExplosion
#@markdown [公式ページ](https://civitai.com/models/1100877/coolguyexplosion?modelVersionId=1236617)
use_cool_guy_explosion = True #@param {type: "boolean"}
if use_cool_guy_explosion:
  !wget https://civitai.com/api/download/models/1236617?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

### start comfyui ###
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python /content/drive/MyDrive/ComfyUI/main.py --dont-print-server

--2025-02-20 03:47:25--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.2.0/cloudflared-linux-amd64.deb [following]
--2025-02-20 03:47:25--  https://github.com/cloudflare/cloudflared/releases/download/2025.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/8176fe8a-cc28-4022-90f5-59d75983238d?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250220%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250220T034725Z&X-Amz-Expires=300&X-Amz-Signature=51fb0db48d7a253523bfd670f66146e6540fc10ca36bcb176f644c7e588c

In [22]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python /content/drive/MyDrive/ComfyUI/main.py --dont-print-server

--2025-02-20 04:20:17--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.2.0/cloudflared-linux-amd64.deb [following]
--2025-02-20 04:20:17--  https://github.com/cloudflare/cloudflared/releases/download/2025.2.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/8176fe8a-cc28-4022-90f5-59d75983238d?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250220%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250220T042017Z&X-Amz-Expires=300&X-Amz-Signature=ae37cc0aca426bce7c5ebd7527eedadf116a4198f28d86ca3ab9d32bb07b

In [25]:
!pip install torchsde

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.4 MB/s eta 0:00:00


In [26]:
import torchsde

In [13]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6

In [18]:
# prompt: localizar e abrir main.py

# Assuming main.py is in the current working directory or a specified path
# If main.py is in the current directory:
!python /content/drive/MyDrive/ComfyUI/main.py

# If main.py is in a different directory (e.g., /content/ComfyUI):
# !python /content/ComfyUI/main.py


[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-02-20 04:14:07.516
** Platform: Linux
** Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/drive/MyDrive/ComfyUI
** ComfyUI Base Folder Path: /content/drive/MyDrive/ComfyUI
** User directory: /content/drive/MyDrive/ComfyUI/user
** ComfyUI-Manager config path: /content/drive/MyDrive/ComfyUI/user/default/ComfyUI-Manager/config.ini
** Log path: /content/drive/MyDrive/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   5.3 seconds: /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12979 MB
pytorch version: 2.5.1+cu121
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using pytorch attention
Traceback (most recent call last):
  File "/content/drive/MyDrive/ComfyUI/mai

In [11]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [24]:
import torchde
print(torch.__version__)

ModuleNotFoundError: No module named 'torchde'

In [16]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [17]:
# Desinstalar CUDA atual
!apt-get --purge remove "*cublas*" "cuda*" "nsight*" "nvidia*"
!apt-get autoremove
!apt-get clean

# Adicionar a chave GPG do CUDA
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/3bf863cc.pub
!sudo apt-key add 3bf863cc.pub

# Adicionar o repositório do CUDA 11.8
!bash -c 'echo "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64 /" > /etc/apt/sources.list.d/cuda.list'
!apt-get update

# Instalar o CUDA 11.8
!apt-get install -y cuda-11-8

# Verificar a versão do CUDA instalada
!nvcc --version


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cublasmp-cuda-11' for glob '*cublas*'
Note, selecting 'cublasmp-cuda-12' for glob '*cublas*'
Note, selecting 'cublasmp0' for glob '*cublas*'
Note, selecting 'libcublasmp0-dev-cuda-11' for glob '*cublas*'
Note, selecting 'libcublasmp0-dev-cuda-12' for glob '*cublas*'
Note, selecting 'libcublasmp0-cuda-11' for glob '*cublas*'
Note, selecting 'libcublasmp0-cuda-12' for glob '*cublas*'
Note, selecting 'libcublas.so.11' for glob '*cublas*'
Note, selecting 'libcublas.so.12' for glob '*cublas*'
Note, selecting 'libcublas-dev-11-0' for glob '*cublas*'
Note, selecting 'libcublas-dev-11-1' for glob '*cublas*'
Note, selecting 'libcublas-dev-11-7' for glob '*cublas*'
Note, selecting 'libcublas-dev-11-8' for glob '*cublas*'
Note, selecting 'libcublas-dev-12-0' for glob '*cublas*'
Note, selecting 'libcublas-dev-12-1' for glob '*cublas*'
Note, selecting 'libcublas-dev-12-2' for glob '*cub

# Nova seção